In [ ]:
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
from langchain_together import ChatTogether
from langchain_community.document_loaders import YoutubeLoader
from langchain.prompts import PromptTemplate
from langchain import LLMChain
import re

# Initialize Flask app
app = Flask(__name__)

# Initialize LLM with your API key from Together.AI
api_key = ["API_KEY"]
llm = ChatTogether(api_key=api_key, temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

# Define a function to check if a URL is a valid YouTube link
def is_youtube_url(url):
    youtube_regex = re.compile(r'(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/.+')
    return youtube_regex.match(url) is not None

# Create a prompt template for summarization
product_description_template = PromptTemplate(
    input_variables=["video_transcript"],
    template="""
    Read through the entire transcript carefully.
    Provide a concise summary of the video's main topic and purpose.
    Extract and list the five most interesting or important points from the transcript. 
    Use clear, engaging language that is accessible to a general audience.
    
    Video transcript: {video_transcript}
    """
)

# Create a function to perform the summarization
def summarise(url):
    try:
        # Load the video transcript using the provided URL
        loader = YoutubeLoader.from_youtube_url(url, add_video_info=False)
        data = loader.load()

        # Run the summarization chain using the transcript data
        chain = LLMChain(llm=llm, prompt=product_description_template)
        summary = chain.invoke({"video_transcript": data[0].page_content})

        return summary['text']
    except Exception as e:
        return f"Error summarizing video: {str(e)}"

# Define a route to handle incoming WhatsApp messages
@app.route('/summary', methods=['POST'])
def whatsapp_summary():
    url = request.form.get('Body')  # Get video URL from incoming message
    print(f"Received URL: {url}")

    if is_youtube_url(url):
        response_text = summarise(url)
    else:
        response_text = "Please check if this is a correct YouTube video URL."

    # Create a Twilio response object
    resp = MessagingResponse()
    msg = resp.message(response_text)
    return str(resp)

# Run the Flask app
if __name__ == '__main__':
    app.run(port=8080)